In [1]:
#initialization
import matplotlib.pyplot as plt
import numpy as np

# importing Qiskit
from qiskit import IBMQ, Aer, assemble, transpile, quantum_info, visualization
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.providers.ibmq import least_busy

# import basic plot tools
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import QFT
from numpy import pi

In [2]:
def createBellPair(qc, qr, bellState):
    if (bellState == 1):
        qc.h(qr[0])
        qc.cx(qr[0], qr[1])
    elif (bellState == 2):
        qc.x(qr[0])
        qc.h(qr[0])
        qc.cx(qr[0], qr[1])
    elif (bellState == 2):
        qc.h(qr[0])
        qc.cx(qr[0], qr[1])
        qc.x(qr[1])
    else:
        qc.x(qr[0])
        qc.h(qr[0])
        qc.cx(qr[0], qr[1])
        qc.x(qr[1])
        
def aliceGates(qc, psi, e, bellState):
    if (bellState == 1):
        qc.cx(psi, e)
        qc.h(psi)
        
def bobGates(qc, e, crz, crx, bellState):
    if (bellState == 1):
        qc.x(e).c_if(crx, 1)
        qc.z(e).c_if(crz, 1)

In [3]:
# Generate the quantum state we wish to teleport
psi = quantum_info.random_statevector(2)
bellState = 1

# Creating the neccesary registers
psiReg = QuantumRegister(1, name="psi")
eReg = QuantumRegister(2, name="e")
crz, crx = ClassicalRegister(1, name="crz"), ClassicalRegister(1, name="crx")
qc = QuantumCircuit(psiReg, eReg, crz, crx)

# Initialize psi
qc.initialize(psi, psiReg)
qc.barrier()

# Generate the bell pair
createBellPair(qc, eReg, bellState)
qc.barrier()

# Alice performs her gates
aliceGates(qc, psiReg, eReg[0], bellState)
qc.barrier()

# Alice performs her measurements and sends them
qc.measure(psiReg, crz)
qc.measure(eReg[0], crx)
qc.barrier()

# Bob performs his gates
bobGates(qc, eReg[1], crz, crx, bellState)

qc.draw()
# Run the circuit and print the state vector of bobs qubit
# sim = Aer.get_backend('aer_simulator')
# qc.save_statevector()
# result = sim.run(qc).result()
# out_vector = result.get_statevector()
# sent_state = []
# for i in range(0, len(out_vector)):
#     if (out_vector[i] != 0):
#         sent_state.append(out_vector[i])

# display(visualization.array_to_latex(psi, prefix="|\\psi\\rangle ="))
# display(visualization.array_to_latex(sent_state, prefix="|\\psi^{Bob}\\rangle ="))

┌───────────────────────────────────────────────┐ ░            ░      »
psi_0: ┤ Initialize(0.24121+0.22122j,0.27731-0.90331j) ├─░────────────░───■──»
       └───────────────────────────────────────────────┘ ░ ┌───┐      ░ ┌─┴─┐»
  e_0: ──────────────────────────────────────────────────░─┤ H ├──■───░─┤ X ├»
                                                         ░ └───┘┌─┴─┐ ░ └───┘»
  e_1: ──────────────────────────────────────────────────░──────┤ X ├─░──────»
                                                         ░      └───┘ ░      »
crz: 1/══════════════════════════════════════════════════════════════════════»
                                                                             »
crx: 1/══════════════════════════════════════════════════════════════════════»
                                                                             »
«       ┌───┐ ░ ┌─┐    ░               
«psi_0: ┤ H ├─░─┤M├────░───────────────
«       └───┘ ░ └╥┘┌─┐ ░               
«  e_0: ──────░──╫─┤M├─░───────────────
«             ░  ║ └╥┘ ░  ┌───┐  ┌───┐ 
«  e_1: ──────░──╫──╫──░──┤ X ├──┤ Z ├─
«             ░  ║  ║  ░  └─╥─┘  └─╥─┘ 
«                ║  ║       ║   ┌──╨──┐
«crz: 1/═════════╩══╬═══════╬═══╡ = 1 ╞
«                0  ║    ┌──╨──┐└─────┘
«crx: 1/════════════╩════╡ = 1 ╞═══════
«                   0    └─────┘